# Getting the most out of NSIDC DAAC data: Discovering, Accessing, and Harmonizing Arctic Remote Sensing Data

Here are the steps you will learn in this tutorial, which will be broken out into several Python Jupyter Notebooks:

**Introduction**
  
   * Learn the basics of Jupyter Lab and Jupyter Notebook
   * Get introduced to the Arctic sea ice case study
   * Explore NSIDC data availability and resources
   * Visualize time and area of interest using OpenAltimetry 
   
**Customize and Access Data**   
   
   * Search for data programmatically by time and area of interest.
   * Determine subsetting, reformatting, and reprojection capabilities for our data of interest.
   * Access and customize data using NSIDC's API service.

**Visualize and Analyze Data**

   * Filter ICESat-2 data by quality flag 
   * Extracting gridded data along ICESat-2 tracks using xarray
   * Plot ICESat-2 tracks against swath and gridded data
   * Plot data with MODIS true color WMS endpoint. 
   
  
 ___

# Introduction to Project Jupyter 

Before we delve into our Arctic sea ice data exploration, let's go over the basics of how to interact with these Jupyter Notebooks and the Jupyter Lab interface, as well as some background on our Jupyter Hub environment.

### The "classic" Jupyter Notebook 

Interactive, reproducible, exploratory, open source

  * Browser integrated computing environment
  * Popular for interactive data analysis
  * Reproducible document formats combined into single notebook (code, markdown, equations, visualizations)
  * Create and share documents that contain live code, equations, visualizations and narrative text.
  * Supports over 100 programming languages 
    
### JupyterLab

Integrated IDE for computational workflows

  * Web-based interactive development environment for Jupyter notebooks, code, and data
  * JupyterLab goes beyond the "classic" Jupyter Notebook by providing a flexble and extensible web application with a set of reusable components
  * All the building blocks for computation put into one integrated interface 
  * Allows multiple notebooks, text editors, terminals, output areas, and custom components using tabs and collapsible sidebars
  
  
### JupyterHub

You are here! For this AGU tutorial we will be working within the NSIDC JupyterHub, powered by Amazon Web Services: https://nsidc-cryohub.org/

No need to download or install any packages. We have preconfigued the environments and data that you'll be working with during the tutorial.

* NSIDC CryoHub will not be accessible after the week of AGU. See README file for instructions for installing and running these tutorials locally. 

## Notebook Basics
The body of a notebook is composed of cells. Each cell contains either markdown, code input, code output, or raw text. Cells can be included in any order and edited at-will, allowing for a large ammount of flexibility for constructing a narrative.

- **Markdown cells** - These are used to build a nicely formatted narrative around the code in the document. 

- **Code cells** - These are used to define the computational code in the document. They come in two forms: the *input cell* where the user types the code to be executed, and the *output cell* which is the representation of the executed code. 

- **Raw cells** - These are used when text needs to be included in raw form, without execution or transformation.


### Execute a cell or selected cells by pressing shift + enter

In [2]:
print('Hello World!')

Hello World!


### Execute a cell and insert new cell below by pressing Alt + Enter

In [ ]:
print('Hello World!')

### Bulk comment and uncomment by selecting code and pressing Ctrl + /

In [1]:
dog = ['Amy', 1, 2.6, True, 3000, ['frisbee', 'ball']]
for i in dog:
    print(i)

Amy
1
2.6
True
3000
['frisbee', 'ball']


### Execute multiple cells or run the entire notebook 

 * Select cells with **shift + Up or shift + Down** and then execute selection with **shift + enter.**
 
 * Run the whole notebook in a single step by clicking on the menu **Cell -> Run All.**
 

___

# Arctic sea ice case study: Fast ice over the East Siberian Sea 

For this tutorial, we're interested in whether or not elevation data from the ICESat-2 mission can help us determine fast ice characteristics over the East Siberian Sea in conjunction with reflectance data.

Outline:

- What is fast ice?
- Describe how fast ice has higher roughness compared to drift ice, and how altimetry and reflectance remote sensing data can help us discern between these sea ice types. Probably best to keep it broad for now, and we'll go into data set details below:
    - Background from Walt: "Fast ice should have less height/freeboard variability - less surface roughness. It also crosses new ice in a flaw lead as the drift ice moves away from the fast ice edge. So there is a good graduation of ice thickness from new ice to thicker first-year ice. So this could show how how well ICESat-2 performs at different ice thicknesses. We could use MODIS reflectance to estimate thin ice thickness. ICESat-2 should provide us with a profile of height across these surfaces."
- Where is our study area?
    - Image courtesy of NASA Worldview
    - Used Worldview to find a clear day on March 23, 2019

![Study Area](https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&TIME=2019-03-23T00:00:00Z&BBOX=-1790464,1137664,-60928,2237440&CRS=EPSG:3413&LAYERS=MODIS_Terra_CorrectedReflectance_TrueColor,Coastlines,Reference_Labels&WRAP=day,x,x&FORMAT=image/png&WIDTH=1689&HEIGHT=1074&ts=1573233401157.png)

Outline, cont'd:

- What are the data sets provided by NSIDC that can help us with this study?
- ICESat-2 overview : Link to icesat-2 home page (https://nsidc.org/data/icesat-2)
    - Instrument overview
    - Sea ice products (we're using ATL03 (photon-level) and ATL07 in particular):
           - From the ICESat-2 Data Sets page, you can find a link to each data set home page: 
                - ATL03: https://nsidc.org/data/atl03 
                - ATL07: https://nsidc.org/data/atl07
                - From that home page, several resources are available, including an online user guide (within the User Guide tab of the landing page):
                    - ATL03: https://nsidc.org/data/atl03?qt-data_set_tabs=3#qt-data_set_tabs
                    - ATL07: https://nsidc.org/data/atl07?qt-data_set_tabs=3#qt-data_set_tabs
                - As well as a data dictionary with every data set variable described in detail: 
                    - ATL03: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03-data-dictionary-v001.pdf 
                    - ATL07: https://nsidc.org/sites/nsidc.org/files/technical-references/ATL07-data-dictionary-v001.pdf
           - Resolution, data structure, etc. 
           - Walt had background on why not to use ATL10 but I don't know if this is too much to include: "ATL10 doesn’t have as much data because of quality flags (see attached). It picks up at ~73.8, with a few data points. I think that’s mainly because ATL10 doesn’t get data within 50 km of land. After those few points, there are only a couple of points until ~74.5. This probably because there are no leads over the fast ice, so it loses its sea level calibration point and can’t get valid freeboard values. Then there is the island and the freeboard values pick up at ~76.3 N. This gap reflects the 50 km land buffer and possibly low quality retrievals over thin ice (e.g., signal to noise is too low for thin ice). So the use case is that ATL07 seems to be getting a signal from the thin ice, but the ATL10 quality/confidence level filters don’t allow retrieval of freeboard, which is appropriate. But one could use the MODIS IST to get freeboard from ATL07 where ATL10 doesn’t have freeboard values, using a simple IST vs. height/freeboard relationship."

![ATLAS beam pattern](https://nsidc.org/sites/nsidc.org/files/images/atlas-beam-pattern.png)
(can we make this smaller??)
         
- MODIS sea ice overview : Link to MODIS home page
    - Instrument overview
    - Sea ice products:
            - Resolution, data structure, etc.
            
            
___

# Visualize ICESat-2 data over time and area of interest

We're looking for a clear day across the coast, fast ice, and on to drift ice. 

Because we're mainly interested in ICESat-2 coverage, we'll use the OpenAltimetry application to easily visualize the coverage over our area of interest, as well as overlay MODIS reflectance imagery to observe cloud cover. 

Annotation I created with our study area:
https://openaltimetry.org/data/icesat2/?annoId=119&shareKey=31a6dd4555dc1b8ccbd84fce02a63d37


